In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pylab as plt

In [ ]:
# Data Set
# https://www.kaggle.com/theeyeschico/vegetable-classification-using-transfer-learning

In [2]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(filters=8, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(224, 224, 3)))
model.add(tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size = (2,2)))
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size = (2,2)))
model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(100, activation='relu', name='Dense1' ))
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Dense(40, activation='relu', name='Dense2' ))
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Dense(15, activation='softmax', name='output'))

In [3]:
img = plt.imread('../input/vegetable-image-dataset/Vegetable Images/train/Broccoli/0026.jpg')
plt.imshow(img)

In [4]:
model.summary()

In [5]:
# model.compile(metrics='accuracy', optimizer = 'adam', loss='sparse_categorical_crossentropy')
model.compile(metrics='accuracy', optimizer = 'adam', loss='categorical_crossentropy')

In [6]:
train_data = ImageDataGenerator(rescale=1./244)
test_data = ImageDataGenerator(rescale=1./244)
training_set = train_data.flow_from_directory('../input/vegetable-image-dataset/Vegetable Images/train/', target_size=(224,224), class_mode='categorical', batch_size=35)
valid_set = train_data.flow_from_directory('../input/vegetable-image-dataset/Vegetable Images/validation/', target_size=(224,224), class_mode='categorical' ,batch_size=35)

In [7]:
early_Stoping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', 
                                min_delta=0, 
                                patience=4, 
                                verbose=0, 
                                mode='auto', 
                                baseline=None, 
                                restore_best_weights=False)

In [8]:
op = model.fit_generator(training_set, epochs=60, validation_data=valid_set, steps_per_epoch=400, callbacks = [early_Stoping])

In [9]:
labels = list(training_set.class_indices.keys())

In [10]:
test_img = tf.keras.preprocessing.image.load_img('../input/vegetable-image-dataset/Vegetable Images/train/Broccoli/0026.jpg', target_size = (224, 224))
test_img = tf.keras.preprocessing.image.img_to_array(test_img)
test_img = np.expand_dims(test_img, axis = 0) # The extra dimension at the beginning of images are generally for batch sizes
result = model.predict(test_img)
print(labels[np.argmax(result[0])])